# This notebook walks through the basics of using TASM.

## Setup: loading a video and detections.

In [ ]:
# Remove previously-stored videos, data.
# TODO: Switch to in-memory db.
import os
import shutil

if os.path.exists('resources'):
    shutil.rmtree('resources')
if os.path.exists('labels.db'):
    os.remove('labels.db')

In [ ]:
VIDEO_PATH = 'data/birds.mp4'
DETECTIONS_PATH = 'data/birds.pkl'

### Load detections.

In [ ]:
import pandas as pd

detections = pd.read_pickle(DETECTIONS_PATH)
detections = detections.astype({'x1': int, 'y1': int, 'x2': int, 'y2': int})
detections.head()

### View detections on a few frames.

In [ ]:
import cv2
import matplotlib.pyplot as plt

def draw_box(img, label, x1, y1, x2, y2):
    pt1 = (x1, y1)
    pt2 = (x2, y2)
    cv2.rectangle(img, pt1, pt2, (255, 0, 0), 2)
    cv2.putText(img, label, (x1, y1 - 5), cv2.FONT_HERSHEY_SIMPLEX, 2, [255, 0, 0], 2)
    return img

num_frames = 5
vc = cv2.VideoCapture(VIDEO_PATH)

for f in range(num_frames):
    ret, frame = vc.read()
    if not ret:
        break
        
    objects = detections[detections.frame == f]
    for i, row in objects.iterrows():
        frame = draw_box(frame, row.label, row.x1, row.y1, row.x2, row.y2)
    
    plt.figure(figsize=(10, 10))
    plt.axis('off')
    plt.imshow(frame); plt.show()

## Loading the video and detections into TASM.

In [ ]:
import os
import tasm

t = tasm.TASM()

# First, load the video without tiling into TASM.
untiled_video_name = 'birds-untiled'
t.store(VIDEO_PATH, untiled_video_name)

# We can also store it with a uniform layout, e.g. 2x2.
rows = 2
cols = 2
t.store_with_uniform_layout(VIDEO_PATH, 'birds-2x2', rows, cols)

In [ ]:
# Add metadata about the video. 
# We could use t.add_metadata(video, label, frame, x1, y1, x2, y2).
# However, since we have all of the metadata available, we can use t.add_bulk_metadata to add all of it at once.
metadata_id = 'birds'
label = 'bird'
metadata_info = [
    tasm.MetadataInfo(metadata_id, r.label, r.frame, r.x1, r.y1, r.x2, r.y2) 
    for _, r in detections.iterrows()
]
t.add_bulk_metadata(metadata_info)

In [ ]:
# Now, we can use the metadata to tile around birds.
tiled_video_name = 'birds-birds'
t.store_with_nonuniform_layout(VIDEO_PATH, tiled_video_name, metadata_id, label)

## Retrieving objects of interest using TASM.

In [ ]:
# First, let's retrieve a few birds from the un-tiled video.
first_frame_inclusive = 0
last_frame_exclusive = 5
untiled_selection = t.select(untiled_video_name, metadata_id, label, first_frame_inclusive, last_frame_exclusive)

# Inspect the returned instances.
num_birds = 0
while True:
    bird = untiled_selection.next()
    if bird.is_empty():
        break
    w = bird.width()
    h = bird.height()
    num_birds += 1
    plt.axis('off')
    plt.imshow(bird.numpy_array()); plt.show()
print(f'Detected {num_birds} birds.')

In [ ]:
# Compare the time to retrieve all of the birds from the untiled video from the tiled video.
# By not specifying first frame and last frame, TASM selects all instances of the object from the video.
import time

start = time.perf_counter()
untiled_selection = t.select(untiled_video_name, metadata_id, label)
num_birds = 0
while True:
    bird = untiled_selection.next()
    if bird.is_empty():
        break
    num_birds += 1
end = time.perf_counter()
print(f'Retrieved {num_birds} birds in {end - start}')

In [ ]:
start = time.perf_counter()
tiled_selection = t.select(tiled_video_name, metadata_id, label)
num_birds = 0
while True:
    bird = tiled_selection.next()
    if bird.is_empty():
        break
    num_birds += 1
end = time.perf_counter()
print(f'Retrieved {num_birds} birds in {end - start}')

## Automatically tiling a video.